# Formula of updating weights during the backpropagations

# The Learning rate tells the optimizer how much step to go down or up

In [ ]:
#################  w_new = w_old - lr*gradient(derivativeOFloss / derivativeOF w)

: 

In [ ]:
################# what is the difference between F.relu and nn.ReLU

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter



import random
import numpy as np

SEED = 55643
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)


device = 'GPU' if torch.cuda.is_available() else 'cpu'


# Hyperparameters are:-- input_size, out_size, kernel_size, learinig_rate

# Parameters are:- weihgts and bias

# ANN

In [ ]:
input_size = 784
hidden_layers = 100
num_classes = 10
lr = 0.001
batch_size = 100

class ArteficialNeuralNetwork(nn.Module):
    
    def __init__(self):
        super(ArteficialNeuralNetwork, self).__init__()
        
        self.lin_1 = nn.Linear(in_features=input_size, out_features=hidden_layers)
        self.relu  = nn.ReLU() # we can also use instead of here inside of the forward func [F.relu(out)]
        self.lin_2 = nn.Linear(in_features=hidden_layers, out_features=num_classes)
        
    def forward(self, x):
        out = self.lin_1(x)
        out = self.relu(out)
        out = self.lin_2(out)
        return out
    
ANN_model = ArteficialNeuralNetwork()
ANN_model

ArteficialNeuralNetwork(
  (lin_1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (lin_2): Linear(in_features=100, out_features=10, bias=True)
)

# CNN
# Be Care full will the kernel_size ,  stride, padding

In [ ]:
#### Parameters
input_size = 1 # it is one cuz we are dealing with white and black color image (gray scale images)
num_classes = 10
num_epoch  = 2  
batch_size = 100
learning_rate = 0.001


class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=input_size, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        # below the 16 comes from the out_channel how about the 7*7 ->  i think it is the height and width of the img
        # it reduce from 28*28 to 7*7 b/c of the operation
        self.fc1 = nn.Linear(in_features=16*7*7, out_features=num_classes) 
        # i think the 

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = x.reshape(-1, 28*28)
        x = self.fc1(x)
        return x
    
CNN_model = CNN().to(device)
CNN_model   

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [2]:
print(28*28)
print(16*7*7)

784
784


# RNN

In [ ]:
# i think RNN take one row in a time not like CNN takes 784 at once(the whole pic)
input_size = 28 
sequence_len = 28 # optional. it might help you during the traing process
hidden_size = 128 # you can chose any number
num_classes = 10
num_layers = 2
num_epochs = 2
batch_size = 100
learning_rate = 0.001

device = 'GPU' if torch.cuda.is_available() else 'cpu'


#### Model
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        ##### i think here below we are initiating the (num_layers and hidden_size) klte layer with 128 nodes
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) # not clear
        #print('print from forward method >>>', len(h0[0][0]), len(h0[1][0]))
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc1(out)
        return out
    
RNN_model = RNN(input_size, hidden_size, num_layers, num_classes)
RNN_model  


RNN(
  (rnn): RNN(28, 128, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=128, out_features=10, bias=True)
)

# BRNN---- Bidirectional 

In [ ]:
input_size = 28 
sequence_len = 28 
hidden_size = 128 # you can chose any number
num_classes = 10
num_layers = 2
batch_size = 100
num_epochs = 2
learning_rate = 0.001

class BRNN(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super(BRNN, self).__init__()
        
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        
    def forward(self, x):
        # why times 2 ---- i think one it goes forward and the second goes backward
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out 
    
        #print("form BRNN...", out.shape) --> torch.Size([100, 10] cuz it has to be passed through the fc
        

BRNN_model = BRNN(num_layers, hidden_size)
BRNN_model

BRNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=10, bias=True)
)

# ========================= Training=========================
# =========================================================

# Be care full on changing the optimizer for each model

In [ ]:
#### loss
loss_func = nn.CrossEntropyLoss()

# optimizer = torch.optim.Adam(ANN_model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(CNN_model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(RNN_model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(BRNN_model.parameters(), lr=learning_rate)

In [ ]:
batch_size = 100

training_data = torchvision.datasets.MNIST('dataMNIST/', 
                                           train=True, 
                                           transform=transforms.ToTensor(), 
                                           #transform=transforms.Normalize(),
                                           download=True)

testing_data  = torchvision.datasets.MNIST('dataMNIST/', 
                                           train=False, 
                                           transform=transforms.ToTensor(), 
                                           #transform=transforms.Normalize(),
                                           download=True)

train_loader = torch.utils.data.DataLoader(dataset=training_data, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(dataset=testing_data, batch_size=batch_size, shuffle=True) 
# we've 10000 test samples and 60000 train samples

# Firts try it with single image always always...........
##### if you see your loss function is 

In [ ]:
# if you see the loss error is getting lower and lower
# this means your model is read to be traind

images, labels = next(iter(train_loader))

for epoch in range(12):
    # 👉👉👉 try it first with on batch #
    images = images.to(device)
    labels = labels.to(device)
    #print(images.shape, labels) #### we got 100 images with 100 labels and 1 means it's black and white(1 depth)
    #torch.Size([100, 1, 28, 28])

    #ANN_model ----- torch.Size([100, 128])
    #images = images.reshape(-1, 28*28).to(device) # only for ANN_model  [input must have 2 dimensions]


    #RNN_model & BRNN_model ---- torch.Size([100, 28, 28])
    images = images.reshape(-1, 28, 28).to(device) # RNN_model & BRNN_model ---- [input must have 3 dimensions]


    #forward
    scores_pred = RNN_model(images)
    loss = loss_func(scores_pred, labels)

    #backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent update step/adam step
    optimizer.step()

    print(f"Loss-erorr: {loss}")

Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117
Loss-erorr: 2.3044614791870117


# Training the model

# Be care full on changing on the images.reshape for each model.....!

In [ ]:
for epoch in range(num_epochs):
    corr = 0
    n_samples = 0
    ix = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        #print(images.shape, labels) #### we got 100 images with 100 labels
        #torch.Size([100, 1, 28, 28])
        
        #ANN_model ----- torch.Size([100, 128])
        #images = images.reshape(-1, 28*28).to(device) # only for NN_model  [input must have 2 dimensions]

        
        #RNN_model & BRNN_model -- torch.Size([100, 28, 28])
        images = images.reshape(-1, 28, 28).to(device) #only for RNN_model & BRNN_model --[input must have 3 dim]
        
        
        #forward
        scores_pred = RNN_model(images)
        loss = loss_func(scores_pred, labels)

        #backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent update step/adam step
        optimizer.step()
        
        _, predictions = torch.max(scores_pred, 1)
        good_pred = (predictions == labels).sum().item()
        
        corr += good_pred
        n_samples += labels.size(0)
        ix += 1
    
        if ix % 100 == 0:
            print(loss)
            
    print(f" >>> Accuracy on Training-data >> : {100 * corr / n_samples}")

tensor(2.3045, grad_fn=<NllLossBackward0>)
tensor(2.3113, grad_fn=<NllLossBackward0>)
tensor(2.2998, grad_fn=<NllLossBackward0>)
tensor(2.3000, grad_fn=<NllLossBackward0>)
tensor(2.3095, grad_fn=<NllLossBackward0>)
tensor(2.3010, grad_fn=<NllLossBackward0>)
 >>> Accuracy on Training-data >> : 12.108333333333333
tensor(2.3020, grad_fn=<NllLossBackward0>)
tensor(2.3025, grad_fn=<NllLossBackward0>)
tensor(2.3032, grad_fn=<NllLossBackward0>)
tensor(2.3103, grad_fn=<NllLossBackward0>)
tensor(2.3108, grad_fn=<NllLossBackward0>)
tensor(2.3010, grad_fn=<NllLossBackward0>)
 >>> Accuracy on Training-data >> : 12.108333333333333


# Testing the model ---> use [ with torch.no_grads ]:

In [ ]:
with torch.no_grad():
    corr = 0
    n_samples = 0
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        #print(images.shape, labels) #### we got 100 images with 100 labels
        #torch.Size([100, 1, 28, 28])

        #ANN_model ----- torch.Size([100, 128])
        #images = images.reshape(-1, 28*28).to(device) # only for NN_model  [input must have 2 dimensions]


        #RNN_model & BRNN_model ---- torch.Size([100, 28, 28])
        images = images.reshape(-1, 28, 28).to(device) # RNN_model & BRNN_model ----[input must have 3 dimensions]


        #forward
        scores_pred = RNN_model(images)

        _, predictions = torch.max(scores_pred, 1)
        good_pred = (predictions == labels).sum().item()

        corr += good_pred
        n_samples += labels.size(0)

    print(f" >>> Accuracy ON Testing-data >> : {100 * corr / n_samples}")

 >>> Accuracy ON Testing-data >> : 12.16
